In [5]:
## Fast Import
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys, os
import pathlib
import time
import math, random
import pprint
import collections
import numbers, string

import yaml
from tqdm import tqdm

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk
from PIL import Image

import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import albumentations as A

np.set_printoptions(precision=3)
curr_path = pathlib.Path(os.getcwd()).absolute()

cards = !echo $SGE_HGR_gpu_card
device = torch.device(f"cuda:{cards[0]}" if torch.cuda.is_available() else 'cpu')
print(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu


# PGL Model + Crit + Optim + Scheduler Setup

In [8]:
from experiments.pgl.pgl_unet3d import UNet3D
from lib.nets.ema import create_ema_model, update_ema_model

model = UNet3D(n_input=1, n_class=1, act='relu')
ema_model = create_ema_model(UNet3D(n_input=1, n_class=1, act='relu'))
model, ema_model = model.to(device), ema_model.to(device)
if True:
    print(f'  * {-1} GPUs, using nn.DataParallel.')
    model = nn.DataParallel(model)
    ema_model = nn.DataParallel(ema_model)

💠 UNet3D-PGL model initiated with n_classes=1, 
   n_input=1, activation=relu, 
   params=19,073,665, trainable_params=19,073,665.
💠 UNet3D-PGL model initiated with n_classes=1, 
   n_input=1, activation=relu, 
   params=19,073,665, trainable_params=19,073,665.
  * -1 GPUs, using nn.DataParallel.


In [30]:
from lib.assess.losses3d import BYOL3d

criterion = BYOL3d()

# Test criterion 
pred = torch.randn((200, 1, 96, 96, 16))
targ = torch.randn((200, 1, 96, 96, 16))

# BYOL 2 - 2 * .. loss
def byol1(pred, targ):
    pred = pred.view(pred.shape[0], -1)
    targ = targ.view(targ.shape[0], -1)
    pred_norm = F.normalize(pred, dim=-1, p=2)
    targ_norm = F.normalize(targ, dim=-1, p=2)
    loss = torch.sum(2 - 2 * (pred_norm * targ_norm).sum(dim=-1)) / pred.shape[0]
    print(loss)
    return loss

def byol2(pred, targ):
    pred = pred.view(pred.shape[0], -1)
    targ = targ.view(targ.shape[0], -1)
    pred_norm = F.normalize(pred, dim=-1, p=2)
    targ_norm = F.normalize(targ, dim=-1, p=2)
    loss = ((pred_norm - targ_norm) ** 2).sum() / pred.shape[0]
    print(loss, '-')
    return loss

times1 = np.zeros(10)
times2 = np.zeros(10)
for i in range(10):
    start = time.time()
    byol1(pred, targ)
    times1[i] = time.time() - start
    
    start = time.time()
    byol2(pred, targ)
    times2[i] = time.time() - start
print(f'1: {times1.mean():.3f}, 2: {times2.mean():.3f}')


💠 Criterion: BYOL3d initiated.
1: 0.089, 2: 0.116


# PGL Data Setup

In [4]:
from experiments.pgl import data_setup

data_setup.get_data_components(None)
